# Writing a Molecular Monte Carlo Simulation

Starting today, make sure you have the functions

1. `calculate_LJ` - written in class
1. `read_xyz` - provided in class
1. `calculate_total_energy` - modified version provided in this notebook written for homework which has cutoff
1. `calculate_distance` - should be the version written for homework which accounts for periodic boundaries.
1. `calculate_tail_correction` - written for homework 


In [1]:
# add imports here
import math
import random

In [26]:
def calculate_total_energy(coordinates, box_length, cutoff):
    """
    Calculate the total energy of a set of particles using the Lennard Jones potential.
    
    Parameters
    ----------
    coordinates : list
        A nested list containing the x, y,z coordinate for each particle
    box_length : float
        The length of the box. Assumes cubic box.
    cutoff : float
        The cutoff length
    
    Returns
    -------
    total_energy : float
        The total energy of the set of coordinates.
    """
    
    total_energy = 0
    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            # Calculate the distance between the particles - exercise.
            dist_ij = calculate_periodic_distance(coordinates[i], coordinates[j], box_length)

            if dist_ij < cutoff:
                # Calculate the pairwise LJ energy
                LJ_ij = calculate_LJ(dist_ij)

                # Add to total energy.
                total_energy += LJ_ij
    return total_energy

def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    return atomic_coordinates, box_length

def calculate_LJ(r_ij):
    """
    The LJ interaction energy between two particles.

    Computes the pairwise Lennard Jones interaction energy based on the separation distance in reduced units.

    Parameters
    ----------
    r_ij : float
        The distance between the particles in reduced units.
    
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.

    Examples
    --------
    >>> calculate_LJ(1)
    0

    """
    
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    
    pairwise_energy = 4 * (r12_term - r6_term)
    
    return pairwise_energy


## Add your group's modified calculate_distance function
def calculate_periodic_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two points. When `box_length` is set, the minimum image convention is used to calculate the distance between the points.

    Parameters
    ----------
    coord1, coord2 : list
        The coordinates of the points, [x, y, z]
    
    box_length : float, optional
        The box length

    Returns
    -------
    distance : float
        The distance between the two points accounting for periodic boundaries
    """
    distance = 0
        
    for i in range(3):
        hold_dist = abs(coord2[i] - coord1[i])
    
        if box_length != None:    
            if hold_dist > box_length/2:
                hold_dist = hold_dist - (box_length * round(hold_dist/box_length))
        distance += math.pow(hold_dist, 2)

    return math.sqrt(distance)

## Add your group's tail correction function
def calculate_tail_correction(n_particles, box_length, cutoff):
    """
    Calculate tail correction
    
    Parameters
    ----------
    box_length: float
        length of box
    
    n_particles: float
        number of particles
        
    cutoff: float
        cutoff distance beyond which interactions are not calculated
        
    Returns
    ---------
    tail_corr: float
        calculated tall correction value
    """
    V=math.pow(box_length,3)
    rc_inv=1/cutoff
    bracket=math.pow(rc_inv,9)*1/3-math.pow(rc_inv,3)
    tail_corr=(8*math.pi*math.pow(n_particles,2)/3/V)*bracket
    return tail_corr

## The Metropolis Criteron
$$ P_{acc}(m \rightarrow n) = \text{min} \left[
		1,e^{-\beta \Delta U}
	\right] $$

In [3]:
def accept_or_reject(delta_U, beta):
    """
    Determines whether a change of a particle in configuration is accepted or rejected based on Metropolis criterion.
    
    Parameters
    ---------
    delta_U: float
        The change in energy for moving system from state m to n.
    beta: float
        1/temperature
        
    Returns
    ---------
    bool
        Whether the move is accepted
    """
    if delta_e <=0.0:
        accept = True
    else:
        #generte a random number on (0,1)
        random_number= random.random()
        p_acc = math.exp(-beta*delta_U)
        
        if random_number < p_acc:
            accept = True
        else:
            accept = False
            
    return accept

In [4]:
# sanity check - test cases
delta_e, beta = -1, 1
assert accept_or_reject(delta_e,beta)

In [5]:
delta_e, beta = 0, 1
print(delta_e)
assert accept_or_reject(delta_e,beta)

0


In [6]:
# To test function with random numbers
# can set random seed

#to set seed
random.seed(0)

In [7]:
random.random()

0.8444218515250481

In [8]:
#test case for positive delta_U
delta_e, beta=1, 1
random.seed(0)
#expected to fail
assert not accept_or_reject(delta_e,beta)

In [9]:
random.seed(1)
random.random()

0.13436424411240122

In [10]:
#test case for positive delta_U
delta_e, beta=1, 1
random.seed(1)
#expected to pass
assert accept_or_reject(delta_e,beta)

In [11]:
#unset seed
random.seed()

In [12]:
def calculate_pair_energy(coordinates, i_particle, box_length, cutoff):
    """
    Calculate the interaction energy of a paritucle with tis environment (all other particles in the system)
    
    Parameters
    ---------
    coordinates : list
        the coordinates for all particles in the system
    i_particle: int
        the particle number for which the calculate the energy
    box_length: float
        the legnth of the simulation box. Assumes cubic box.
    cutoff: float
        the simulation cutoff, Beyond this distances, interactions are not calulated
    
    Returns
    ---------
    float
        The pairwise interaction energy of the ith particles with all other particles in the system
    
    """
    e_total = 0.0
    i_position = coordinates[i_particle]
    
    num_atoms = len(coordinates)
    
    for j_particle in range(num_atoms):
            if j_particle != i_particle:
                j_position = coordinates[j_particle]
                r_ij=calculate_periodic_distance(i_position, j_position, box_length)
                
                if r_ij<cutoff:
                    e_pair = calculate_LJ(r_ij)
                    e_total += e_pair
    return e_total

In [13]:
test_coords = [[0,0,0],[0,0,2**(1/6)],[0,0,2*2**(1/6)]]
calculate_LJ(2**(1/6))

-1.0

In [15]:
#what do i expect the results to be for particle index 1?  -2
assert calculate_pair_energy(test_coords, 1, 10, 3) == -2
#what do i expect the results to be for particle index 0 with a cutoff of 2? -1
assert calculate_pair_energy(test_coords, 0, 10, 2) == -1

In [16]:
enery=calculate_periodic_distance([0,0,2**(1/6)],[0,0,0], 10)
calculate_LJ(enery)

-1.0

In [17]:
enery=calculate_periodic_distance([0,0,2**(1/6)],[0,0,2*2**(1/6)], 10)
calculate_LJ(enery)

-1.0

In [18]:
calculate_pair_energy(test_coords, 1, 10, 3)

-2.0

In [19]:
enery=calculate_periodic_distance([0,0,2**(1/6)],[0,0,0], 10)
calculate_LJ(enery)

-1.0

In [20]:
enery=calculate_periodic_distance([0,0,2*2**(1/6)],[0,0,0], 10)
calculate_LJ(enery)

-0.031005859374999993

In [21]:
test_coords

[[0, 0, 0], [0, 0, 1.122462048309373], [0, 0, 2.244924096618746]]

In [22]:
calculate_pair_energy(test_coords, 0, 10, 2)

-1.0

# Monte Carlo Loop

In [31]:
#read or generate intial coordinates
coordinates, box_length = read_xyz('lj_sample_configurations/lj_sample_config_periodic1.txt')

#set simulation parameters
reduced_temperature = 0.9
num_steps= 5000
max_displacement = 0.1
cutoff = 3
freq = 1000

#Caculated quatities
beta = 1/reduced_temperature
num_particles = len(coordinates)

#energy calculation
total_energy = calculate_total_energy (coordinates, box_length, cutoff)
print (total_energy)
tail_correction = calculate_tail_correction(num_particles, box_length, cutoff)
print (tail_correction)

total_energy += tail_correction



-4351.540194543858
-198.48888374415657


In [32]:
for step in range (num_steps):
    #1. randomly pick one of the paricles
    random_particle=random.randrange(num_particles)
    
    #2. calculate the interaction energy of the selected particle with the stsem and store this value
    current_energy=calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
    
    #3. generate a random x,y,z displacement
    x_rand= random.uniform(-max_displacement, max_displacement)
    y_rand= random.uniform(-max_displacement, max_displacement)
    z_rand= random.uniform(-max_displacement, max_displacement)
    
    #4. modify the coordinate of the indexed particle by generated dispalcements
    coordinates[random_particle][0] += x_rand
    coordinates[random_particle][1] += y_rand
    coordinates[random_particle][2] += z_rand
    
    #5. calculate the interaction energy of the moved particle with the system and store this value.
    proposed_energy=calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
    delta_energy=proposed_energy-current_energy
    
    #6. Calcualte if we accept the move based on energy difference
    accept=accept_or_reject(delta_energy,beta)
    
    #7. if accepted, update energy. if rejected move back particle coordintes
    if accept:
        total_energy += delta_energy
    else: 
        coordinates[random_particle][0] -= x_rand
        coordinates[random_particle][1] -= y_rand
        coordinates[random_particle][2] -= z_rand
    #8. print the energy if step is multiple of freq
    if step % freq ==0:
        print(step, total_energy/num_particles)

0 -5.688671885044659
1000 -5.710306995108846
2000 -5.728009645021001
3000 -5.67229586054138
4000 -5.668544408606924
